In this section we will investigate and Compare different join strategies by joining the fact_int_sales and dim_product
- First we Will cash the two tables, since this notebook will be part of and ETL Job so these tables and joins will be reused, hence cashing is a good idea.

In [0]:
#Load Data
options = {
    "header": "true",
    "inferSchema": "true",
    "delimiter": ";"
} 
dim_product =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_product/")
fact_int_sales =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/fact_int_sales/")
dim_sales_terr =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_sales_terr/")
dim_curreny = spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_curreny/")  

In [0]:
fact_sales = fact_int_sales.cache()
dim_product = dim_product.cache()
fact_sales.count()  # Force caching (Since Cashing is a lazy evaluatioin)
dim_product.count()
join_key = "ProductKey"

when Joining
    - Spark reads data in partitions, and when performing joins, it shuffles data to ensure that rows with matching keys are placed on the same executor.
    - **Example**: Given `fact_int_sales` and `city` tables, both with a `dim_product` column, Spark distributes data across executors. After reading, Spark shuffles the data to match rows with the same `ProductKey` for the join.

## Broadcast Join

In [0]:
from pyspark.sql.functions import broadcast

broadcast_join = fact_sales.join(
    broadcast(dim_product), 
    join_key,
    how="left"
)


In [0]:
display(broadcast_join.limit(10))

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,ListPrice,Size,SizeRange,Weight,DaysToManufacture,ProductLine,DealerPrice,Class,Style,ModelName,EnglishDescription,FrenchDescription,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status
310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-29,2011-01-10,2011-01-05,BK-R93R-62,2,LB,CM,"Road-150 Red, 62","Carretera: 150, roja, 62","Vélo de route 150 rouge, 62","21712942,00",1,Red,100,75,3578.27,62,60-62 CM,15,4,R,2146962,H,U,Road-150,"This bike is ridden by race winners. Developed with the Adventure Works Cycles professional race team, it has a extremely light heat-treated aluminum frame, and steering that allows precision control.","Ce vélo est destiné aux champions cyclistes. Mis au point par une équipe cycliste professionnelle, ce vélo possède un cadre en aluminium traité à chaud extrêmement léger et un guidon qui permet une conduite très précise.",此自行车曾为赛车手夺冠立下汗马功劳。与 Adventure Works Cycles 专业赛车队联合设计，它的铝制车架极其轻巧且经过热处理，操纵装置可以实现精准控制。,إنها الدراجة التي يقودها الفائزون بسباقات الدراجات. ونظرًا لأنه تم تطويرها عن طريق فريق سباق المحترفين Adventure Works Cycles، فقد تم تصميم هيكلها من الألومنيوم الخفيف المُعالج حراريًا، وتم تزويدها بنظام قيادة يسمح بالتحكم الدقيق في الدراجة.,"אופניים אלה משמשים את מנצחי התחרויות. לאופניים, שפותחו יחד עם צוות המרוצים של Adventure Works Cycles (אופני חוויה), יש מסגרת אלומיניום קלה ביותר שהוקשחה בחום והיגוי המאפשר שליטה מדויקת.",จักรยานคู่ใจของนักแข่งระดับแชมเปี้ยน ได้รับการพัฒนาโดยทีมแข่ง Adventure Works Cycles ใช้เฟรมอลูมิเนียมผ่านความร้อนซึ่งเบาเป็นพิเศษ และการบังคับที่แม่นยำ,"Dieses Fahrrad wird von Rennprofis gefahren. Es wurde gemeinsam mit dem Profirennteam von Adventure Works Cycles entwickelt und hat einen extrem leichten, hitzebehandelten Aluminiumrahmen und eine hochpräzise Steuerung.",レース ウィナーも愛用する機種です。Adventure Works Cycles プロフェッショナル レーシング チームと共に開発されたものであり、非常に軽量な熱処理アルミニウム フレームと、精密な制御が可能なステアリングを備えています。,"Bu bisiklet yarışçılar tarafından kullanılmaktadır. Adventure Works Cycles profesyonel yarış takımıyla birlikte geliştirilen bu bisiklet, son derece hafif, tavlanmış alüminyum kadroya ve hassas kontrol sağlayan bir gidona sahiptir.",2011-07-01T00:00:00.000+0000,2007-12-28 00:00:00.000,NULL
346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,BK-M82S-44,1,LB,CM,"Mountain-100 Silver, 44","Montaña: 100, plateada, 44","VTT 100 argent, 44","19121544,00",1,Silver,100,75,3399.99,44,42-46 CM,21.13,4,M,2039994,H,U,Mountain-100,"Top-of-the-line competition mountain bike. Performance-enhancing options include the innovative HL Frame, super-smooth front suspension, and traction for all terrain.","VTT de compétition haut de gamme. Plusieurs options d'amélioration des performances : cadre HL, suspension avant particulièrement souple et traction adaptée à tous les terrains.",高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地形的出色牵引力。,دراجة سباقات مخصصة للقيادة في الجبال من أعلى طراز. تتضمن خيارات تحسين الأداء هيكل HL Frame الإبداعي، والتعليق الأمامي ذي السلاسة الفائقة، وقوة الجر المناسبة لجميع أنواع الأراضي.,"אופני הרים תחרותיים מהשורה הראשונה. אפשרויות לשיפור ביצועים הכוללות HL Frame (מסגרת (HL, שיכוך קדמי חלק במיוחד ואחיזת כביש המתאימה לכל סוגי השטח.",สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด้วยองค์ประกอบสำหรับประสิทธิภาพสูงสุด เช่น เฟรม HL นวัตกรรมใหม่ ระบบกันกระเทือนด้านหน้าที่นุ่มนวลเป็นพิเศษ และยางที่

## Sort-Merge Join 

In [0]:
sort_merge_join = fact_sales.join(
    dim_product, 
    join_key,
    how="left"
)

In [0]:
display(sort_merge_join.limit(10)) 

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,ListPrice,Size,SizeRange,Weight,DaysToManufacture,ProductLine,DealerPrice,Class,Style,ModelName,EnglishDescription,FrenchDescription,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status
310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-29,2011-01-10,2011-01-05,BK-R93R-62,2,LB,CM,"Road-150 Red, 62","Carretera: 150, roja, 62","Vélo de route 150 rouge, 62","21712942,00",1,Red,100,75,3578.27,62,60-62 CM,15,4,R,2146962,H,U,Road-150,"This bike is ridden by race winners. Developed with the Adventure Works Cycles professional race team, it has a extremely light heat-treated aluminum frame, and steering that allows precision control.","Ce vélo est destiné aux champions cyclistes. Mis au point par une équipe cycliste professionnelle, ce vélo possède un cadre en aluminium traité à chaud extrêmement léger et un guidon qui permet une conduite très précise.",此自行车曾为赛车手夺冠立下汗马功劳。与 Adventure Works Cycles 专业赛车队联合设计，它的铝制车架极其轻巧且经过热处理，操纵装置可以实现精准控制。,إنها الدراجة التي يقودها الفائزون بسباقات الدراجات. ونظرًا لأنه تم تطويرها عن طريق فريق سباق المحترفين Adventure Works Cycles، فقد تم تصميم هيكلها من الألومنيوم الخفيف المُعالج حراريًا، وتم تزويدها بنظام قيادة يسمح بالتحكم الدقيق في الدراجة.,"אופניים אלה משמשים את מנצחי התחרויות. לאופניים, שפותחו יחד עם צוות המרוצים של Adventure Works Cycles (אופני חוויה), יש מסגרת אלומיניום קלה ביותר שהוקשחה בחום והיגוי המאפשר שליטה מדויקת.",จักรยานคู่ใจของนักแข่งระดับแชมเปี้ยน ได้รับการพัฒนาโดยทีมแข่ง Adventure Works Cycles ใช้เฟรมอลูมิเนียมผ่านความร้อนซึ่งเบาเป็นพิเศษ และการบังคับที่แม่นยำ,"Dieses Fahrrad wird von Rennprofis gefahren. Es wurde gemeinsam mit dem Profirennteam von Adventure Works Cycles entwickelt und hat einen extrem leichten, hitzebehandelten Aluminiumrahmen und eine hochpräzise Steuerung.",レース ウィナーも愛用する機種です。Adventure Works Cycles プロフェッショナル レーシング チームと共に開発されたものであり、非常に軽量な熱処理アルミニウム フレームと、精密な制御が可能なステアリングを備えています。,"Bu bisiklet yarışçılar tarafından kullanılmaktadır. Adventure Works Cycles profesyonel yarış takımıyla birlikte geliştirilen bu bisiklet, son derece hafif, tavlanmış alüminyum kadroya ve hassas kontrol sağlayan bir gidona sahiptir.",2011-07-01T00:00:00.000+0000,2007-12-28 00:00:00.000,NULL
346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,BK-M82S-44,1,LB,CM,"Mountain-100 Silver, 44","Montaña: 100, plateada, 44","VTT 100 argent, 44","19121544,00",1,Silver,100,75,3399.99,44,42-46 CM,21.13,4,M,2039994,H,U,Mountain-100,"Top-of-the-line competition mountain bike. Performance-enhancing options include the innovative HL Frame, super-smooth front suspension, and traction for all terrain.","VTT de compétition haut de gamme. Plusieurs options d'amélioration des performances : cadre HL, suspension avant particulièrement souple et traction adaptée à tous les terrains.",高档竞赛山地车。性能得到进一步增强，包括创新的 HL 车架、极其平稳的前悬架以及适用于所有地形的出色牵引力。,دراجة سباقات مخصصة للقيادة في الجبال من أعلى طراز. تتضمن خيارات تحسين الأداء هيكل HL Frame الإبداعي، والتعليق الأمامي ذي السلاسة الفائقة، وقوة الجر المناسبة لجميع أنواع الأراضي.,"אופני הרים תחרותיים מהשורה הראשונה. אפשרויות לשיפור ביצועים הכוללות HL Frame (מסגרת (HL, שיכוך קדמי חלק במיוחד ואחיזת כביש המתאימה לכל סוגי השטח.",สุดยอดจักรยานภูเขาสำหรับการแข่งขัน สมบูรณ์แบบด้วยองค์ประกอบสำหรับประสิทธิภาพสูงสุด เช่น เฟรม HL นวัตกรรมใหม่ ระบบกันกระเทือนด้านหน้าที่นุ่มนวลเป็นพิเศษ และยางที่

## Bucketed Join

In [0]:
# Bucket tables first (if not already)
fact_sales_bucketed = fact_sales.repartitionByRange(100, join_key)
dim_products_bucketed = dim_product.repartitionByRange(100, join_key)

bucketed_join = fact_sales_bucketed.join(
    dim_products_bucketed, 
    join_key,
    how="left"
)

In [0]:
display(bucketed_join.limit(10))

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,ListPrice,Size,SizeRange,Weight,DaysToManufacture,ProductLine,DealerPrice,Class,Style,ModelName,EnglishDescription,FrenchDescription,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status
214,20121228,20130109,20130104,16313,1,100,8,SO51180,4,34.99,34.99,13.09,13.09,34.99,2.8,0.8748,2012-12-28,2013-01-09,2013-01-04,HL-U509-R,31,NULL,NULL,"Sport-100 Helmet, Red","Casco deportivo: 100, rojo","Casque sport 100, rouge","130863,00",1,Red,4,3,34.99,NULL,NA,NULL,0,S,20994,NULL,NULL,Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.","Légère, aérée, taille unique, avec une visière amovible.",通用型透气良好且轻便，带有自合型帽沿。,ملائمة بشكل عام، وجيدة التهوية، وخفيفة الوزن بقناع واق من الشمس مزود بخاصية الالتصاق التلقائي.,"מידה אונברסלית, מאווררת היטב, קלת-משקל, עם מצחייה מתחברת.",แว่นกันลมขนาดสากล ระบายอากาศได้ดี น้ำหนักเบา,"Einheitsgröße, atmungsaktiv, leicht, aufsetzbarer Blendschutz.",軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"Her bisiklete kolayca takılabilen, aerodinamik, hafif dikiz aynası.",2013-07-01T00:00:00.000+0000,NULL,Current
214,20121228,20130109,20130104,12132,1,100,7,SO51181,4,34.99,34.99,13.09,13.09,34.99,2.8,0.8748,2012-12-28,2013-01-09,2013-01-04,HL-U509-R,31,NULL,NULL,"Sport-100 Helmet, Red","Casco deportivo: 100, rojo","Casque sport 100, rouge","130863,00",1,Red,4,3,34.99,NULL,NA,NULL,0,S,20994,NULL,NULL,Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.","Légère, aérée, taille unique, avec une visière amovible.",通用型透气良好且轻便，带有自合型帽沿。,ملائمة بشكل عام، وجيدة التهوية، وخفيفة الوزن بقناع واق من الشمس مزود بخاصية الالتصاق التلقائي.,"מידה אונברסלית, מאווררת היטב, קלת-משקל, עם מצחייה מתחברת.",แว่นกันลมขนาดสากล ระบายอากาศได้ดี น้ำหนักเบา,"Einheitsgröße, atmungsaktiv, leicht, aufsetzbarer Blendschutz.",軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"Her bisiklete kolayca takılabilen, aerodinamik, hafif dikiz aynası.",2013-07-01T00:00:00.000+0000,NULL,Current
214,20121229,20130110,20130105,12390,1,100,8,SO51191,4,34.99,34.99,13.09,13.09,34.99,2.8,0.8748,2012-12-29,2013-01-10,2013-01-05,HL-U509-R,31,NULL,NULL,"Sport-100 Helmet, Red","Casco deportivo: 100, rojo","Casque sport 100, rouge","130863,00",1,Red,4,3,34.99,NULL,NA,NULL,0,S,20994,NULL,NULL,Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.","Légère, aérée, taille unique, avec une visière amovible.",通用型透气良好且轻便，带有自合型帽沿。,ملائمة بشكل عام، وجيدة التهوية، وخفيفة الوزن بقناع واق من الشمس مزود بخاصية الالتصاق التلقائي.,"מידה אונברסלית, מאווררת היטב, קלת-משקל, עם מצחייה מתחברת.",แว่นกันลมขนาดสากล ระบายอากาศได้ดี น้ำหนักเบา,"Einheitsgröße, atmungsaktiv, leicht, aufsetzbarer Blendschutz.",軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"Her bisiklete kolayca takılabilen, aerodinamik, hafif dikiz aynası.",2013-07-01T00:00:00.000+0000,NULL,Current
214,20121229,20130110,20130105,11241,1,100,7,SO51192,2,34.99,34.99,13.09,13.09,34.99,2.8,0.8748,2012-12-29,2013-01-10,2013-01-05,HL-U509-R,31,NULL,NULL,"Sport-100 Helmet, Red","Casco deportivo: 100, rojo","Casque sport 100, rouge","130863,00",1,Red,4,3,34.99,NULL,NA,NULL,0,S,20994,NULL,NULL,Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.","Légère, aérée, taille unique, avec une visière amovible.",通用型透气良好且轻便，带有自合型帽沿。,ملائمة بشكل عام، وجيدة التهوية، وخفيفة الوزن بقناع واق من الشمس مزود بخاصية الالتصاق التلقائي.,"מידה אונברסלית, מאווררת היטב, קלת-משקל, עם מצחייה מתחברת.",แว่นกันลมขนาดสากล ระบายอากาศได้ดี น้ำหนักเบา,"Einheitsgröße, atmungsaktiv, leicht, aufsetzbarer B